In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import os

# load results

In [6]:
def load_data(dataset: str) -> dict:
    """
    Load data from pickle files for a specific dataset.
    
    Args:
        dataset (str): Name of the dataset or specific subdirectory
    
    Returns:
        dict: A dictionary with full file paths as keys and loaded data as values
    """
    base_dir = "../results/datasets/real_world_data"
    full_path = os.path.join(base_dir, dataset)
    
    data_dict = {}
    
    # Walk through the directory tree starting from the specified dataset path
    for root, dirs, files in os.walk(full_path):
        for file in files:
            # Check if the file is a pickle file
            if file.endswith('.pkl') or file.endswith('.p'):
                full_file_path = os.path.join(root, file)
                print(full_file_path)
                
                try:
                    # Load the pickle file
                    with open(full_file_path, "rb") as f:
                        data = pickle.load(f)
                    
                    # Store in the dictionary with full path as key
                    data_dict[full_file_path] = data
                
                except (IOError, pickle.UnpicklingError) as e:
                    print(f"Error loading {full_file_path}: {e}")
    
    return data_dict

In [7]:
data_dictionary = load_data("wine")

../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/0.p
../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/16.p
../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/18.p
../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/9.p
../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/22.p
../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/11.p
../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/17.p
../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/12.p
../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/28.p
../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/2.p
../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/14.p
../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/21.p
../resu

/home/thomas/miniconda3/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 1.4.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/thomas/miniconda3/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.4.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/13.p
../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/24.p
../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/23.p
../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/10.p
../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/25.p
../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/6.p
../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/7.p
../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/29.p
../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/15.p
../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/4.p
../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/8.p
../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/19.p
../resul

In [4]:
data_dictionary.keys()

dict_keys(['../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/0.p', '../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/16.p', '../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/18.p', '../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/9.p', '../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/22.p', '../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/11.p', '../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/17.p', '../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/12.p', '../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/28.p', '../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/2.p', '../results/datasets/real_world_data/wine/activation=relu/partial_contrib=True/14.p', '../results/datasets/real_world_data/wine/acti